In [41]:
from os.path import join
import copy

import numpy as np
import pandas as pd
import ast

from sklearn.metrics import fbeta_score

data_dir = "D:/MLiP/data/"

In [8]:
def labels_one_hot(labels):
    
    onehot_label = [0.] * 228
    for label in labels:
        onehot_label[int(label) - 1] = 1.
    
    return onehot_label

In [3]:
predictions_df = pd.DataFrame.from_csv('validation_prediction.csv')
predictions = predictions_df.values
print(predictions.shape)

C:\Users\laure\Anaconda3\envs\neuralsnets\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


(9897, 228)


In [12]:
ground_truth = pd.DataFrame.from_csv(join(data_dir, "csv/valid_labels.csv"))
labels_gt = []
for i in range(9897):
    row = ground_truth.iloc[i, :]
    label = labels_one_hot(ast.literal_eval(row['labelId']))
    labels_gt.append(label)
labels_gt = np.array(labels_gt)
print(labels_gt.shape)

C:\Users\laure\Anaconda3\envs\neuralsnets\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


(9897, 228)


In [76]:
def get_optimal_thresholds(true, pred, epochs=10):
    
    # Assert true and pred have same shape
    dims = true.shape
    assert(dims == pred.shape)

    # Initialize values to be filled in, thresholds start at 0.1
    best_f1 = 0.
    best_threshold = np.array([0.5] * 228)
    
    # Loop through epochs
    for i in range(epochs):
        
        # Loop through labels
        for j in range(dims[1]):
            
            print('Fine-tuning label:', j+1)
            
            # Variable to determine when to move on to next label
            improved = 2
            
            # Loop to find optimal threshold for label j
            for k in np.linspace(0.1, 1, 10):
                
                if improved == 0:
                    print('Stopped fine-tuning label ' + str(j+1) + ' after ' + str(int(k * 10 - 1)) + ' tries.\n')
                    break
                
                # Create copy of thresholds to mute in the loop
                thresholds = copy.copy(best_threshold)
                
                # Set new threshold
                thresholds[j] = k
                
                # Threshold predictions
                thresholded_pred = np.zeros(dims)
                for l in range(dims[0]):
                    thresholded_pred[l] += pred[l] > thresholds

                # Calculate 
                f1 = np.mean(np.array([fbeta_score(true[x], thresholded_pred[x], beta=1) for x in range(dims[0])]))

                # Update score if necessary
                if f1 > best_f1:
                    best_f1 = f1
                    best_threshold = copy.copy(thresholds)
                    print('F1-score improved to:', best_f1)
                else:
                    improved -= 1
        
    return best_f1, best_theshold

In [ ]:
score, thresh = get_optimal_thresholds(labels_gt, predictions, epochs=3)

Fine-tuning label: 1


C:\Users\laure\Anaconda3\envs\neuralsnets\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


F1-score improved to: 0.4635488255016373
Stopped fine-tuning label 1 after 3 tries.
Fine-tuning label: 2
Stopped fine-tuning label 2 after 2 tries.
Fine-tuning label: 3
F1-score improved to: 0.46361162182087495
Stopped fine-tuning label 3 after 3 tries.
Fine-tuning label: 4
F1-score improved to: 0.4638202611282842
Stopped fine-tuning label 4 after 3 tries.
Fine-tuning label: 5
F1-score improved to: 0.46382567724018536
F1-score improved to: 0.46383244680279184
F1-score improved to: 0.4638690228927173
Stopped fine-tuning label 5 after 5 tries.
Fine-tuning label: 6
F1-score improved to: 0.46394960760500864
Stopped fine-tuning label 6 after 3 tries.
Fine-tuning label: 7
Stopped fine-tuning label 7 after 2 tries.
Fine-tuning label: 8
Stopped fine-tuning label 8 after 2 tries.
Fine-tuning label: 9
Stopped fine-tuning label 9 after 2 tries.
Fine-tuning label: 10
F1-score improved to: 0.46409513080297204
Stopped fine-tuning label 10 after 3 tries.
Fine-tuning label: 11
Stopped fine-tuning labe

In [ ]:
print(score)
print(thresh)

In [19]:
x = np.zeros(5)
a = np.array([0.3, 0.5, 0.7, 0.9, 0.6])
b = np.array([0.5, 0.5, 0.5, 0.5, 0.7])

x += (a > b)
print(a > b)
print(x)

[False False  True  True False]
[0. 0. 1. 1. 0.]


In [20]:
# Create dataframes from ground truth and predictions
#df_true = pd.DataFrame(true)
#df_pred = pd.DataFrame(predictions)


In [65]:
print(np.linspace(0.1, 1, 10))

[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
